In [2]:
import requests
import re
import math
import os
import random
import time
import unicodedata  # 자음/모음 분리현상 방지용 모듈
import urllib
import urllib.request
import pandas as pd
import numpy as np
#from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

DB = pd.read_excel("C:/Users/PC/Desktop/부산광역시 RPA/기업DB_뉴스_1차필터.xlsx", sheet_name = "1차필터")
key_word = pd.read_excel("C:/Users/PC/Desktop/부산광역시 RPA/기업DB_뉴스_1차필터.xlsx", sheet_name = "키워드")["키워드"]
company_name = DB["기업명(가공)"]

#기업코드, "(주)", "주식회사" 제거
#company_name = []

#for i in company_list:
#    i = i.replace(i[-8:], "").replace("(주)", "").replace("주식회사", "")
#    company_name.append(i)

#저장할 파일 경로
f_dir = "C://Users//PC//Desktop//"
#저장할 파일 이름
sec_name = "기업"
query_txt = "news"

n = time.localtime()
s1 = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

#os.makedirs(f_dir+s1+'-'+query_txt+'-'+sec_name)
#os.chdir(f_dir+s1+'-'+query_txt+'-'+sec_name)

ff_dir = f_dir+s1+'-'+query_txt+'-'+sec_name
fc_name = f_dir+s1+'-'+query_txt+'-'+sec_name+'\\'+s1+'-'+query_txt+'-'+sec_name+'.csv'

#driver 세팅
chrome_driver_path = 'C:/Users/PC/Desktop/부산광역시 RPA/chromedriver.exe'
options = webdriver.ChromeOptions()

#user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
#options.add_argument("app_version=" +user_agent)
#options.add_argument("--proxy-server=socks5://127.0.0.1:9150")
#UserAgent 랜덤으로 생성
#ua = UserAgent(verify_ssl = False)
#userAgent = ua.random
#생성한 UserAgent 값을 옵션에 추가
#options.add_argument(f"user-agent={userAgent}")

options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument("disable-blink-features=AutomationControlled")
#options.add_argument('headless') #팝업창 안뜨게 제어
options.add_argument('window-size=1920x1080')
#options.add_argument("disable-gpu")
args = ["hide_console", ]
#options.add_argument("--disable-extensions");
options.add_argument("disable-infobars");
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(chrome_driver_path, options=options, service_args=args)


base_url = 'https://search.naver.com/search.naver?where=news&query=%EC%9D%B8%EC%8A%A4%ED%85%8D&sm=tab_opt&sort=1&photo=3&field=0&pd=3&ds=2018.01.01&de=2022.12.31&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Add%2Cp%3Afrom20180101to20221231&is_sug_officeid=0' 
driver.set_window_size(1124, 850)
driver.get(base_url)
#time.sleep(5)
driver.maximize_window()
#WebDriverWait(driver, 200).until(EC.presence_of_element_located((By.XPATH, "/html/body/div/div[2]/section[2]/div/div[1]/div[2]/div[1]/div[1]/div/module/div[1]")))

count = 1

for name, num in zip(company_name, range(0, len(company_name))):
    try:
        search = driver.find_element(By.NAME,'query')
        search.clear()
        search.send_keys(name)
        search.send_keys("\n")

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        page1_news = soup.find('div', 'group_news').find_all('li')
        
        try:
            driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[2]/div/div/a[2]').click()
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            page2_news = soup.find('div', 'group_news').find_all('li')   
            news_list = page1_news + page2_news
        except:
            news_list = page1_news   
    except:
        continue
    time.sleep(0.1)

    for news, col in zip(news_list, range(144,164)):
        
        #기사 출처
        news_url = news.find_all('a')[4]['href']
        
        driver.get(news_url)
        time.sleep(0.5)
        
        page_html = driver.page_source
        page_soup = BeautifulSoup(page_html, 'html.parser')
        
        try:
            if page_soup.find("h1", "erms_h").get_text() == "페이지를 찾을 수 없습니다.":
                continue
        except:
            pass
        
        #기사날짜
        try:
            news_date = page_soup.find("span", "media_end_head_info_datestamp_time _ARTICLE_DATE_TIME").get_text()
        except AttributeError:
            #연예기사
            if page_soup.find('span', 'author') != None:
                news_date = page_soup.find('span', 'author').get_text().split(" ")[1]
            #스포츠기사
            else:
                news_date = page_soup.find("div", "info").get_text().split(" ")[1]
        
        #기사내용    
        try:
            contents = page_soup.find('div', 'newsct_article').get_text()
        except AttributeError:
            #연예기사
            if page_soup.find('div', 'article_body') != None:
                contents = page_soup.find('div', 'article_body').get_text()
            #스포츠기사
            else:
                contents = page_soup.find('div', 'news_end font1 size3').get_text()
            
        contents2 = []
        for i in re.sub("\n|\t", "", contents).split("."):
            for j in key_word:
                if j in i:
                    contents2.append(i)
                else:
                    continue
                
        contents3 = []
        for i in contents2:
            if i not in contents3:
                contents3.append(i)
        
        if contents3 == []:
            result = "-"
        else:
            result = {"contents" : contents3,
                      "date" : news_date,
                      "url" : news_url}
        result = np.array(result).astype(object)  
        
        #DB.at[company_name[company_name == name].index[0], col] = result
        DB.at[num, col] = result     
        
    page_url = 'https://search.naver.com/search.naver?where=news&query=%EC%9D%B8%EC%8A%A4%ED%85%8D&sm=tab_opt&sort=1&photo=3&field=0&pd=3&ds=2018.01.01&de=2022.12.31&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Add%2Cp%3Afrom20180101to20221231&is_sug_officeid=0' 
    driver.get(page_url)
    time.sleep(0.5)
    
    #진행현황
    if count % 100 == 0:
        print(count, "개 진행중")
    count += 1

driver.close()

DB.iloc[:,144:164] = DB.iloc[:,144:164].fillna("-")

for i, j in zip(range(144,164), range(1, 21)):
    DB.rename(columns = {i : "기사" + str(j)}, inplace = True)
DB.to_csv(fc_name, encoding = "utf-8-sig", index = False)

C:\Users\PC\AppData\Local\Temp\ipykernel_10472\1473189541.py:67: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver_path, options=options, service_args=args)


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	Ordinal0 [0x00E78CE3+2067683]
	Ordinal0 [0x00E0CBC1+1625025]
	Ordinal0 [0x00D0B7CD+571341]
	Ordinal0 [0x00CF2E1A+470554]
	Ordinal0 [0x00D5AA2B+895531]
	Ordinal0 [0x00D6ACC6+961734]
	Ordinal0 [0x00D57156+880982]
	Ordinal0 [0x00D2FF1D+720669]
	Ordinal0 [0x00D30F5F+724831]
	GetHandleVerifier [0x0112F162+2771330]
	GetHandleVerifier [0x01121145+2713957]
	GetHandleVerifier [0x00F0CC0A+533546]
	GetHandleVerifier [0x00F0B970+528784]
	Ordinal0 [0x00E14F4C+1658700]
	Ordinal0 [0x00E198B8+1677496]
	Ordinal0 [0x00E199A5+1677733]
	Ordinal0 [0x00E24CC4+1723588]
	BaseThreadInitThunk [0x76016BD9+25]
	RtlGetFullPathName_UEx [0x77AB8FD2+1218]
	RtlGetFullPathName_UEx [0x77AB8F9D+1165]


In [5]:
company_name[3477]

'프롬바이오'

In [ ]:
#1B72643EA0DC57CA60DB2C9CA5A0A060BF147836E725B0173DB6EEC003

In [3]:
DB

,구분,업종,기업명(가공),기업명(기업코드),대표자명,설립일자,사업자번호,법인번호,기업형태,업종코드,...,총자산회전율,속한산업 총자산회전율,상대적.8,매출채권회전율,속한산업 매출채권회전율,상대적.9,보고서 검색 건 수,뉴스검색 건 수,144,145
0,PWC,C,인스텍,(주)인스텍(F37511),이용문,2004년01월01일/2000년07월01일,301-81-73161,150111-0077574,"중소기업,외감",C26211,...,1.91회,10.27회,0.97회,우수,6.44회,우수,0건,0건,{'contents': ['국내 벤처기업의 다중소재를 활용한 금속 3D프린팅 기술이...,{'contents': ['금속 3D 프린터 기업 인스텍의 적층 제조 기술이 의료·...
1,P_확장,G,아이오앤코코리아,(주)아이오앤코코리아(GD6192),전재훈,2015년04월09일/-,712-86-00053,110111-5692317,"중소기업,외감",G47911,...,3.06회,6.89회,0.95회,우수,12.77회,열위,0건,0건,NaN,NaN
2,NaN,J,플리토,(주)플리토(J46180),이정수,2012년08월24일/2019년07월17일,215-87-72878,110111-4945246,"중소기업,코스닥상장",J63120,...,0.37회,9.79회,0.40회,열위,7.89회,우수,0건,0건,NaN,NaN
3,PWC,C,오뚜기,(주)오뚜기(310301),함영준/황성만,1971년06월03일/1994년08월12일,138-81-03238,134111-0002258,"중견기업,상장",C10730,...,1.18회,15.06회,0.92회,우수,12.50회,우수,7건,1건,NaN,NaN
4,PWC,C,CJ제일제당,CJ제일제당(주)(494292),손경식/최은석,2007년09월04일/2007년09월28일,104-86-09535,110111-3748253,"대기업,상장",C10720,...,0.67회,25.32회,0.74회,열위,17.57회,우수,51건,6건,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3589,PWC,H,용마로지스,용마로지스(주)(614190),이종철,1979년02월01일/-,211-86-40630,110111-0245369,"중견기업,외감",H49301,...,1.41회,7.31회,1.40회,우수,7.71회,열위,0건,0건,NaN,NaN
3590,P_확장,H,정양산업,정양산업(주)(130338),정현철,1987년04월05일/-,603-81-08552,180111-0072429,"중소기업,외감",H52102,...,0.50회,8.10회,0.24회,우수,9.03회,열위,0건,0건,NaN,NaN
3591,P_확장,H,정일글로벌,정일글로벌(주)(I84032),최재영,2013년01월08일/-,602-81-60313,180111-0841725,"중소기업,외감",H52102,...,0.32회,5.00회,0.24회,우수,9.03회,열위,0건,0건,NaN,NaN
3592,PWC,H,한국로지스풀,한국로지스풀(주)(434592),서병륜/서용기,2002년06월01일/-,105-86-40937,110111-2539033,"중견기업,외감",H52992,...,2.21회,8.01회,2.16회,우수,8.80회,열위,0건,0건,NaN,NaN
